# Settings

Check GPU

In [ ]:
!nvidia-smi

In [ ]:
import os

main_folder = "outputs"
os.mkdir("/" + main_folder)

In [ ]:
!pip install diffusers==0.3.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

from google.colab import output
from huggingface_hub import notebook_login

output.enable_custom_widget_manager()
notebook_login()

Download and load pipelines

In [ ]:
import torch
from torch import autocast
import string, random
from time import sleep
import inspect
import warnings
from typing import List, Optional, Union
from tqdm.auto import tqdm
from IPython.display import clear_output 
import requests
import json
import numpy
import time
import contextlib
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64encode
import cv2
import math

from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionPipeline

device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

basic_pipe = StableDiffusionPipeline.from_pretrained(
    model_path, 
    revision="fp16", 
    torch_dtype=torch.float16, 
    use_auth_token=True)

basic_pipe = basic_pipe.to(device)

img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
img2img_pipe = img2img_pipe.to(device)


def dummy_checker(images, **kwargs): return images, False
img2img_pipe.safety_checker = dummy_checker
basic_pipe.safety_checker = dummy_checker

In [ ]:
def imshow(name, img):
  """Put frame as <img src="data:image/jpg;base64,...."> """

  js = Javascript('''
  async function showImage(name, image, width, height) {
    img = document.getElementById(name);
    if(img == null) {
      img = document.createElement('img');
      img.id = name;
      document.body.appendChild(img);
    }
    img.src = image;
    img.width = width;
    img.height = height;
  }
  ''')

  height, width = img.shape[:2]

  ret, data = cv2.imencode('.jpg', img)   # compress array of pixels to JPG data
  data = b64encode(data)                  # encode base64
  data = data.decode()                    # convert bytes to string
  data = 'data:image/jpg;base64,' + data  # join header ("data:image/jpg;base64,") and base64 data (JPG)

  display(js)
  eval_js(f'showImage("{name}", "{data}", {width}, {height})')  # run JavaScript code to put image (JPG as string base64) in <img>
                                          # `name` and `data` in needs `" "` to send it as text, not as name of variabe.

In [ ]:
def display_custom(pil_image):
  pil_image.save(folder_name + "/img" + str(math.floor(time.time())) + ".jpg")
  frame1 = cv2.cvtColor(numpy.array(pil_image), cv2.COLOR_RGB2BGR)
  frame1 = cv2.resize(src=frame1, dsize=(0,0), fx=2, fy=2)
  imshow("temp", frame1)

Random.org endpooint integration

In [ ]:
api_key = "xxx"

In [ ]:
random_json = {
    "jsonrpc": "2.0",
    "method": "generateStrings",
    "params": {
        "apiKey": api_key,
        "n": 1,
        "length": 10,
        "characters": "abcdefghijklmnopqrstuvwxyz0123456789 ",
        "replacement": True
    },
    "id": 42
}

headers = {'Content-type': 'application/json','Content-Length': '200', 'Accept': 'application/json'}

data=json.dumps(random_json)

def get_random_string():
  response = requests.post(
    url='https://api.random.org/json-rpc/2/invoke',
    data=data,
    headers=headers
    )
  return(json.loads(response.text)['result']['random']['data'][0])


In [ ]:
max_int = 2147483647
random_int = {
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "params": {
        "apiKey": api_key,
        "n": 1,
        "min": 0,
        "max": 200000000,
        "replacement": True
    },
    "id": 42
}

headers = {'Content-type': 'application/json','Content-Length': '200', 'Accept': 'application/json'}

data_int=json.dumps(random_int)

def get_random_int():
  response = requests.post(
    url='https://api.random.org/json-rpc/2/invoke',
    data=data_int,
    headers=headers
    )
  print(json.loads(response.text))
  return(int(json.loads(response.text)['result']['random']['data'][0]))


# Stream

In [ ]:
n_images = 100
init_image_steps = 25
experiment_name = "test"

In [ ]:
prompt = get_random_string()
from IPython.utils import io

folder_name = "/" + main_folder + "/" + experiment_name + "_" + str(math.floor(time.time()))
os.mkdir(folder_name)

# pseudo = ''.join(random.choice(string.ascii_lowercase) for _ in range(int(random.uniform(1,15))))

with autocast(device):  
  with io.capture_output() as captured:
    image = basic_pipe(prompt, num_inference_steps=init_image_steps).images[0] 

display_custom(image)

for i in range(n_images):
  # prompt = get_random_string()

  # generator = torch.Generator("cuda").manual_seed(1024)
  # image = basic_pipe(prompt, num_inference_steps=init_image_steps).images[0] 

  with autocast(device):  
    #with io.capture_output() as captured:
      generator = torch.Generator("cuda").manual_seed(get_random_int())
      image = basic_pipe(prompt, num_inference_steps=init_image_steps, generator = generator).images[0] 
      #image = basic_pipe(prompt, num_inference_steps=25).images[0] 
   
  display_custom(image)

Post processing

# 4 Grid Static

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    grid.save(folder_name + "/img" + str(math.floor(time.time())) + ".jpg")
    return grid

In [ ]:
prompt = [get_random_string()] * 4
from IPython.utils import io

folder_name = "/" + main_folder + "/" + experiment_name + "_4grid_" + str(math.floor(time.time()))
os.mkdir(folder_name)

with autocast(device):  
  with io.capture_output() as captured:
    images = basic_pipe(prompt, num_inference_steps=init_image_steps).images

grid = image_grid(images, rows=2, cols=2)
grid


# Outro


In [ ]:
import shutil
shutil.make_archive("/" + main_folder, 'zip', "/" + main_folder)